# نمونه‌ای از عامل رزرو هتل و پرواز

این راه‌حل به شما کمک می‌کند بلیت پرواز و هتل رزرو کنید. سناریو شامل یک سفر از فرودگاه هیترو لندن (LHR) در تاریخ ۲۰ فوریه ۲۰۲۴ به فرودگاه جان اف کندی نیویورک (JFK) و بازگشت در تاریخ ۲۷ فوریه ۲۰۲۵ است. پرواز در کلاس اقتصادی و فقط با خطوط هوایی بریتیش ایرویز انجام می‌شود. من می‌خواهم در یک هتل هیلتون در نیویورک اقامت داشته باشم. لطفاً هزینه‌های پرواز و هتل را ارائه دهید.


# راه‌اندازی سرویس Azure AI Agent و دریافت اطلاعات پیکربندی از **.env**

### **.env**

یک فایل .env ایجاد کنید

فایل **.env** شامل رشته اتصال سرویس Azure AI Agent، مدل مورد استفاده توسط AOAI، و همچنین API سرویس جستجوی Google، ENDPOINT و غیره است.

- **AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME** = "نام استقرار مدل سرویس Azure AI Agent شما"

[**NOTE**] شما به مدلی با محدودیت نرخ 100,000 (تعداد توکن‌ها در دقیقه) و محدودیت نرخ 600 (تعداد درخواست‌ها در دقیقه) نیاز دارید.

  می‌توانید مدل را در Azure AI Foundry - Model and Endpoint دریافت کنید.

- **AZURE_AI_AGENT_PROJECT_CONNECTION_STRING** = "رشته اتصال پروژه سرویس Azure AI Agent شما"

  می‌توانید رشته اتصال پروژه را در نمای کلی پروژه خود در صفحه پورتال AI Foundry پیدا کنید.

- **SERPAPI_SEARCH_API_KEY** = "کلید API جستجوی SERPAPI شما"
- **SERPAPI_SEARCH_ENDPOINT** = "نقطه پایانی جستجوی SERPAPI شما"

برای دریافت نام استقرار مدل و رشته اتصال پروژه سرویس Azure AI Agent، باید سرویس Azure AI Agent را ایجاد کنید. توصیه می‌شود از [این قالب](https://portal.azure.com/#create/Microsoft.Template/uri/https%3A%2F%2Fraw.githubusercontent.com%2Ffosteramanda%2Fazure-agent-quickstart-templates%2Frefs%2Fheads%2Fmaster%2Fquickstarts%2Fmicrosoft.azure-ai-agent-service%2Fstandard-agent%2Fazuredeploy.json) برای ایجاد مستقیم آن استفاده کنید. (***توجه:*** سرویس Azure AI Agent در حال حاضر در یک منطقه محدود تنظیم شده است. توصیه می‌شود به [این لینک](https://learn.microsoft.com/en-us/azure/ai-services/agents/concepts/model-region-support) مراجعه کنید تا منطقه را تنظیم کنید.)

Agent نیاز به دسترسی به SERPAPI دارد. توصیه می‌شود با استفاده از [این لینک](https://serpapi.com/searches) ثبت‌نام کنید. پس از ثبت‌نام، می‌توانید یک کلید API و ENDPOINT منحصربه‌فرد دریافت کنید.


# تنظیمات

برای اجرای این نوت‌بوک، باید مطمئن شوید که کتابخانه‌های مورد نیاز را با اجرای `pip install -r requirements.txt` نصب کرده‌اید.


In [ ]:
from semantic_kernel import __version__

__version__

نسخه هسته معنایی شما باید حداقل ۱.۲۷.۲ باشد.


لطفاً فایل .env خود را بارگذاری کنید و منابع را تنظیم کنید. اطمینان حاصل کنید که کلیدها و تنظیمات خود را اضافه کرده‌اید و یک فایل .env محلی ایجاد کرده‌اید.


In [ ]:
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# ورود به Azure

اکنون باید وارد Azure شوید. یک ترمینال باز کنید و دستور زیر را اجرا کنید:

```bash
az login
```

این دستور از شما می‌خواهد که اطلاعات کاربری Azure خود را وارد کنید تا سرویس Azure AI Agent بتواند به درستی کار کند.


# توضیحات:
این یک متغیر است که کلید API را برای دسترسی به سرویس API صفحه نتایج موتور جستجو (SERP) ذخیره می‌کند. کلید API یک شناسه منحصر به فرد است که برای احراز هویت درخواست‌های مرتبط با حساب شما استفاده می‌شود.

هدف: هدف این خط ذخیره کلید API در یک متغیر است تا بتوان از آن برای احراز هویت درخواست‌ها به سرویس SERP API استفاده کرد. کلید API برای دسترسی به سرویس و انجام جستجوها ضروری است.
چگونه یک کلید API برای SERP دریافت کنیم: برای دریافت کلید API برای SERP، مراحل کلی زیر را در https://serpapi.com دنبال کنید (مراحل دقیق ممکن است بسته به سرویس خاص SERP API که استفاده می‌کنید متفاوت باشد):

انتخاب یک سرویس SERP API: چندین سرویس SERP API موجود است، مانند SerpAPI، Google Custom Search JSON API و دیگران. سرویسی را انتخاب کنید که بهترین نیازهای شما را برآورده کند.

ثبت‌نام برای یک حساب کاربری: به وب‌سایت سرویس SERP API انتخابی بروید و برای یک حساب کاربری ثبت‌نام کنید. ممکن است نیاز باشد اطلاعات اولیه‌ای ارائه دهید و آدرس ایمیل خود را تأیید کنید.

ایجاد یک کلید API: پس از ثبت‌نام، وارد حساب کاربری خود شوید و به بخش API یا داشبورد بروید. گزینه‌ای برای ایجاد یا تولید یک کلید API جدید پیدا کنید.
کلید API را در فایل .env خود کپی کنید.


In [ ]:
SERP_API_KEY='SERPAPI_SEARCH_API_KEY'

# توضیح:
BASE_URL: این یک متغیر است که آدرس پایه برای نقطه پایانی API جستجوی SERP را ذخیره می‌کند. نام متغیر BASE_URL به عنوان یک قرارداد استفاده می‌شود تا نشان دهد این آدرس نقطه شروع برای ارسال درخواست‌های API است.
'https://serpapi.com/search':

این رشته آدرس واقعی است که به متغیر BASE_URL اختصاص داده شده است. این آدرس نشان‌دهنده نقطه پایانی برای انجام جستجوها با استفاده از API جستجوی SERP است.

# هدف:
هدف این خط تعریف یک ثابت است که آدرس پایه برای API جستجوی SERP را نگه می‌دارد. این آدرس به عنوان نقطه شروع برای ساخت درخواست‌های API جهت انجام عملیات جستجو استفاده خواهد شد.

# استفاده:
با تعریف آدرس پایه در یک متغیر، می‌توانید به راحتی آن را در سراسر کد خود هر زمان که نیاز به ارسال درخواست به API جستجوی SERP دارید، استفاده کنید. این کار کد شما را قابل نگهداری‌تر می‌کند و خطر خطاهایی که از سخت‌کدنویسی آدرس در چندین مکان ناشی می‌شود را کاهش می‌دهد. مثال فعلی https://serpapi.com/search?engine=bing است که از API جستجوی Bing استفاده می‌کند. API‌های مختلفی را می‌توان در https://Serpapi.com انتخاب کرد.


In [ ]:
BASE_URL = 'https://serpapi.com/search?engine=bing'

# توضیحات:

این بخش محل قرارگیری کد پلاگین شما است.

تعریف کلاس: `class BookingPlugin`: یک کلاس به نام BookingPlugin تعریف می‌کند که شامل متدهایی برای رزرو هتل‌ها و پروازها است.

متد رزرو هتل:

- `@kernel_function(description="booking hotel")`: یک دکوریتور که عملکرد را به عنوان یک تابع کرنل برای رزرو هتل توصیف می‌کند.
- `def booking_hotel(self, query: Annotated[str, "The name of the city"], check_in_date: Annotated[str, "Hotel Check-in Time"], check_out_date: Annotated[str, "Hotel Check-out Time"]) -> Annotated[str, "Return the result of booking hotel information"]:`: یک متد برای رزرو هتل تعریف می‌کند که شامل پارامترهای توضیح داده شده و نوع بازگشتی است.

این متد یک دیکشنری از پارامترها برای درخواست رزرو هتل ایجاد می‌کند و یک درخواست GET به API SERP ارسال می‌کند. وضعیت پاسخ را بررسی کرده و در صورت موفقیت، اطلاعات هتل را بازمی‌گرداند، یا در صورت شکست، مقدار None را برمی‌گرداند.

متد رزرو پرواز:

- `@kernel_function(description="booking flight")`: یک دکوریتور که عملکرد را به عنوان یک تابع کرنل برای رزرو پرواز توصیف می‌کند.
- `def booking_flight(self, origin: Annotated[str, "The name of Departure"], destination: Annotated[str, "The name of Destination"], outbound_date: Annotated[str, "The date of outbound"], return_date: Annotated[str, "The date of Return_date"]) -> Annotated[str, "Return the result of booking flight information"]:`: یک متد برای رزرو پرواز تعریف می‌کند که شامل پارامترهای توضیح داده شده و نوع بازگشتی است.

این متد دیکشنری‌هایی از پارامترها برای درخواست‌های پرواز رفت و برگشت ایجاد می‌کند و درخواست‌های GET به API SERP ارسال می‌کند. وضعیت پاسخ را بررسی کرده و اطلاعات پرواز را به رشته نتیجه اضافه می‌کند در صورت موفقیت، یا در صورت شکست یک پیام خطا چاپ می‌کند. این متد رشته‌ای شامل اطلاعات پرواز را بازمی‌گرداند.


In [ ]:
import requests

from typing import Annotated

from semantic_kernel.functions import kernel_function

# Define Booking Plugin
class BookingPlugin:
    """Booking Plugin for customers"""

    @kernel_function(description="booking hotel")
    def booking_hotel(
        self, 
        query: Annotated[str, "The name of the city"], 
        check_in_date: Annotated[str, "Hotel Check-in Time"], 
        check_out_date: Annotated[str, "Hotel Check-out Time"],
    ) -> Annotated[str, "Return the result of booking hotel information"]:
        """
        Function to book a hotel.
        Parameters:
        - query: The name of the city
        - check_in_date: Hotel Check-in Time
        - check_out_date: Hotel Check-out Time
        Returns:
        - The result of booking hotel information
        """

        # Define the parameters for the hotel booking request
        params = {
            "engine": "google_hotels",
            "q": query,
            "check_in_date": check_in_date,
            "check_out_date": check_out_date,
            "adults": "1",
            "currency": "GBP",
            "gl": "uk",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request to the SERP API
        response = requests.get(BASE_URL, params=params)

        # Check if the request was successful
        if response.status_code == 200:
            # Parse the response content as JSON
            response = response.json()
            # Return the properties from the response
            return response["properties"]
        else:
            # Return None if the request failed
            return None

    @kernel_function(description="booking flight")
    def booking_flight(
        self, 
        origin: Annotated[str, "The name of Departure"], 
        destination: Annotated[str, "The name of Destination"], 
        outbound_date: Annotated[str, "The date of outbound"], 
        return_date: Annotated[str, "The date of Return_date"],
    ) -> Annotated[str, "Return the result of booking flight information"]:
        """
        Function to book a flight.
        Parameters:
        - origin: The name of Departure
        - destination: The name of Destination
        - outbound_date: The date of outbound
        - return_date: The date of Return_date
        - airline: The preferred airline carrier
        - hotel_brand: The preferred hotel brand
        Returns:
        - The result of booking flight information
        """
        
        # Define the parameters for the outbound flight request
        go_params = {
            "engine": "google_flights",
            "departure_id": "destination",
            "arrival_id": "origin",
            "outbound_date": "outbound_date",
            "return_date": "return_date",
            "currency": "GBP",
            "hl": "en",
            "airline": "airline",
            "hotel_brand": "hotel_brand",
            "api_key": "SERP_API_KEY"
        }
 
        print(go_params)

        # Send the GET request for the outbound flight
        go_response = requests.get(BASE_URL, params=go_params)

        # Initialize the result string
        result = ''

        # Check if the outbound flight request was successful
        if go_response.status_code == 200:
            # Parse the response content as JSON
            response = go_response.json()
            # Append the outbound flight information to the result
            result += "# outbound \n " + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Define the parameters for the return flight request
        back_params = {
            #"engine": "google_flights",
            "departure_id": destination,
            "arrival_id": origin,
            "outbound_date": outbound_date,
            "return_date": return_date,
            "currency": "GBP",
            "hl": "en",
            "api_key": SERP_API_KEY
        }

        # Send the GET request for the return flight
        back_response = requests.get(BASE_URL, params=back_params)

        # Check if the return flight request was successful
        if back_response.status_code == 200:
            # Parse the response content as JSON
            response = back_response.json()
            # Append the return flight information to the result
            result += "\n # return \n" + str(response)
        else:
            # Print an error message if the request failed
            print('error!!!')

        # Print the result
        print(result)

        # Return the result
        return result


# توضیحات:
وارد کردن دستورات: وارد کردن ماژول‌های لازم برای اعتبارنامه‌های Azure، عامل هوش مصنوعی، محتوای پیام‌های چت، نقش نویسنده، و تزئین‌کننده توابع هسته.

مدیریت زمینه غیرهمزمان:  
`async with (DefaultAzureCredential() as creds, AzureAIAgent.create_client(credential=creds, conn_str="...") as client,)`: این کد یک مدیریت زمینه غیرهمزمان تنظیم می‌کند تا اعتبارنامه‌های Azure را مدیریت کرده و یک کلاینت عامل هوش مصنوعی ایجاد کند.

نام عامل و دستورالعمل‌ها:  
- `AGENT_NAME = "BookingAgent"`: نام عامل را تعریف می‌کند.  
- `AGENT_INSTRUCTIONS = """..."""`: دستورالعمل‌های دقیق برای عامل فراهم می‌کند تا درخواست‌های رزرو را مدیریت کند.

ایجاد تعریف عامل:  
`agent_definition = await client.agents.create_agent(...)`: یک تعریف عامل با مدل، نام و دستورالعمل‌های مشخص ایجاد می‌کند.

ایجاد عامل AzureAI:  
`agent = AzureAIAgent(...)`: یک عامل AzureAI با استفاده از کلاینت، تعریف عامل، و افزونه تعریف‌شده ایجاد می‌کند.

ایجاد رشته:  
`thread: AzureAIAgentThread | None = None`: یک رشته برای عامل ایجاد می‌کند. لازم نیست ابتدا یک رشته ایجاد شود - اگر مقدار `None` ارائه شود، یک رشته جدید در اولین فراخوانی ایجاد شده و به عنوان بخشی از پاسخ بازگردانده می‌شود.

ورودی‌های کاربر:  
`user_inputs = ["..."]`: لیستی از ورودی‌های کاربر را تعریف می‌کند که عامل باید پردازش کند.

در بلوک `finally`، رشته و عامل حذف می‌شوند تا منابع پاکسازی شوند.


# احراز هویت

کلاس `DefaultAzureCredential` بخشی از Azure SDK برای پایتون است. این کلاس یک روش پیش‌فرض برای احراز هویت با سرویس‌های Azure ارائه می‌دهد. این کلاس تلاش می‌کند با استفاده از چندین روش به ترتیب مشخصی احراز هویت کند، مانند متغیرهای محیطی، هویت مدیریت‌شده و اعتبارنامه‌های Azure CLI.

عملیات غیرهمزمان: ماژول aio نشان می‌دهد که کلاس DefaultAzureCredential از عملیات غیرهمزمان پشتیبانی می‌کند. این به این معناست که می‌توانید از آن همراه با asyncio برای انجام درخواست‌های احراز هویت غیرمسدودکننده استفاده کنید.


In [ ]:
# Import necessary modules
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents import AzureAIAgent, AzureAIAgentSettings, AzureAIAgentThread

ai_agent_settings = AzureAIAgentSettings.create()

# Azure AI Setting
async with (
     DefaultAzureCredential() as creds,
    AzureAIAgent.create_client(
        credential=creds,
        conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
    ) as client,
):    
    
    # Define the agent's name and instructions
    AGENT_NAME = "BookingAgent"
    AGENT_INSTRUCTIONS = """
    You are a booking agent, help me to book flights or hotels.

    Thought: Understand the user's intention and confirm whether to use the reservation system to complete the task.

    Action:
    - If booking a flight, convert the departure name and destination name into airport codes.
    - If booking a hotel or flight, use the corresponding API to call. Ensure that the necessary parameters are available. If any parameters are missing, use default values or assumptions to proceed.
    - If it is not a hotel or flight booking, respond with the final answer only.
    - Output the results using a markdown table:
    - For flight bookings, separate the outbound and return contents and list them in the order of Departure_airport Name | Airline | Flight Number | Departure Time | Arrival_airport Name | Arrival Time | Duration | Airplane | Travel Class | Price (USD) | Legroom | Extensions | Carbon Emissions (kg).
    - For hotel bookings, list them in the order of Properties Name | Properties description | check_in_time | check_out_time | prices | nearby_places | hotel_class | gps_coordinates.
    """

    # Create agent definition with the specified model, name, and instructions
    agent_definition = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=AGENT_NAME,
        instructions=AGENT_INSTRUCTIONS,
    )

    # Create the AzureAI Agent using the client and agent definition
    agent = AzureAIAgent(
        client=client,
        definition=agent_definition,
        plugins=[BookingPlugin()]
    )

    # Create a new thread for the agent
    # If no thread is provided, a new thread will be
    # created and returned with the initial response
    thread: AzureAIAgentThread | None = None

    # This is your prompt for the activity or task you want to complete 
    # Define user inputs for the agent to process we have provided some example prompts to test and validate 
    user_inputs = [
        # "Can you tell me the round-trip air ticket from  London to New York JFK aiport, the departure time is February 17, 2025, and the return time is February 23, 2025"
        # "Book a hotel in New York from Feb 20,2025 to Feb 24,2025"
        "Help me book flight tickets and hotel for the following trip London Heathrow LHR Feb 20th 2025 to New York JFK returning Feb 27th 2025 flying economy with British Airways only. I want a stay in a Hilton hotel in New York please provide costs for the flight and hotel"
        # "I have a business trip from London LHR to New York JFK on Feb 20th 2025 to Feb 27th 2025, can you help me to book a hotel and flight tickets"
    ]

    try:
        # Process each user input
        for user_input in user_inputs:
            print(f"# User: '{user_input}'")
            # Get the agent's response for the specified thread
            response = await agent.get_response(
                messages=user_input,
                thread=thread,
            )
            thread = response.thread
            # Print the agent's response
            print(f"{response.name}: '{response.content}'")
    finally:
        # Clean up by deleting the thread and agent
        await thread.delete() if thread else None
        await client.agents.delete_agent(agent.id)


---

**سلب مسئولیت**:  
این سند با استفاده از سرویس ترجمه هوش مصنوعی [Co-op Translator](https://github.com/Azure/co-op-translator) ترجمه شده است. در حالی که ما تلاش می‌کنیم دقت را حفظ کنیم، لطفاً توجه داشته باشید که ترجمه‌های خودکار ممکن است شامل خطاها یا نادرستی‌ها باشند. سند اصلی به زبان اصلی آن باید به عنوان منبع معتبر در نظر گرفته شود. برای اطلاعات حساس، توصیه می‌شود از ترجمه حرفه‌ای انسانی استفاده کنید. ما مسئولیتی در قبال سوء تفاهم‌ها یا تفسیرهای نادرست ناشی از استفاده از این ترجمه نداریم.
